# Santa's Sleigh

The year 2020 is coming to an end and Santa is preparing for Christmas. However, this year the Elves made too many gifts and Santa can't take all of them with him at once: the total of his sleigh has to be below below a certain threshold so that his reindeer are still able to pull the sleigh around the world. Therefore, Santa needs to carefully choose the gifts to take so that his sleigh is not overloaded and the amount of happiness he brings to the world is as large as possible. (The remaining gifts will be delivered by other means.) Santa asked you for help with this task.

The table below lists all of Santa's gifts together with their weight and the amount of happiness they bring to the world. Your task is to choose a set of gifts such that the combined weight is below 250 kg, which is the weight limit of the sleigh, and the total happiness is maximized.

<table class="center">
    <tr>
        <th style="text-align:center">Item </th>
        <th style="text-align:center">Weight [kg]</th>
        <th style="text-align:center">Happiness </th>
        <th style="text-align:center">Amount </th>
    </tr>
    <tr>
        <td style="text-align:center">Teddy Bear</td>
        <td style="text-align:center">0.5</td>
        <td style="text-align:center">7</td>
        <td style="text-align:center">90</td>
    </tr>
    <tr>
        <td style="text-align:center">LEGO Saturn V Rocket</td>
        <td style="text-align:center">2.5</td>
        <td style="text-align:center">32</td>
        <td style="text-align:center">20</td>
    </tr>
    <tr>
        <td style="text-align:center">Mountain Bike</td>
        <td style="text-align:center">16.5</td>
        <td style="text-align:center">230</td>
        <td style="text-align:center">6</td>
    </tr>
    <tr>
        <td style="text-align:center">LEGO Mindstorms RobotKit</td>
        <td style="text-align:center">2</td>
        <td style="text-align:center">28</td>
        <td style="text-align:center">20</td>
    </tr>
    <tr>
        <td style="text-align:center">Barbie Malibu House</td>
        <td style="text-align:center">5</td>
        <td style="text-align:center">70</td>
        <td style="text-align:center">10</td>
    </tr>
    <tr>
        <td style="text-align:center">Playmobil RC Freight Train</td>
        <td style="text-align:center">4</td>
        <td style="text-align:center">50</td>
        <td style="text-align:center">15</td>
    </tr>
</table>

## Task 1. DP Solution

<b>Your task:</b> Solve the problem using dynamic programming.

In [1]:
import numpy as np

def knapsack_dp(values,weights,n_items,capacity,return_all=False):
    table = np.zeros((n_items+1,capacity+1),dtype=np.float32)
    keep = np.zeros((n_items+1,capacity+1),dtype=np.float32)
    for i in range(1,n_items+1):
        for w in range(0,capacity+1):
            wi = weights[i-1] # weight of current item
            vi = values[i-1] # value of current item
            if (wi <= w) and (vi + table[i-1,w-wi] > table[i-1,w]):
                table[i,w] = vi + table[i-1,w-wi]
                keep[i,w] = 1
            else:
                table[i,w] = table[i-1,w]
    picks = []
    K = capacity
    for i in range(n_items,0,-1):
        if keep[i,K] == 1:
            picks.append(i)
            K -= weights[i-1]
    picks.sort()
    picks = [x-1 for x in picks] # change to 0-index
    if return_all:
        max_val = table[n_items,capacity]
        return picks,max_val
    return picks


items = {"Teddy Bear" : [0.5,7,90,0],
         "LEGO Saturn V Rocket" : [2.5,32,20,0],
         "Mountain Bike" : [16.5,230,6,0],
         "LEGO Mindstorms RobotKit" : [2,28,20,0],
         "Barbie Malibu House" : [5,70,10,0],
         "Playmobil RC Freight Train" : [4,50,15,0]
}

#We change our bounded Knapsack problem to a 0-1 Knapsack problem, by creating one entrie for each item we have
#e.g we will create 90 entries with value 7 and weight 1 for the 90 Teddy Bears we have
#we double our weights to get integer values for our weights, the result will not change!
val=[] #values of happiness
wt=[] #weights of the items
n=0  #number of total items we choose from
for arr in items.values():
    for i in range(arr[2]):
        val.append(arr[1])
        wt.append(int(2*arr[0]))
    n+=arr[2]
#our Capacity or Weight allowed (again doubled preserve the right result)
W=500

#a list of the indexes of the items we choose
picks = knapsack_dp(val,wt,n,W)

#convert our picks back to the number of items we have to take of each type
it=0
curr_max = list(items.values())[it][2]
for i in range(len(picks)):
    if picks[i] < curr_max:
        list(items.values())[it][3]+=1
    else:
        it += 1
        curr_max += list(items.values())[it][2]
        list(items.values())[it][3]+=1


print("Santa has to pack:")
for m in items.items():
    print(str(m[1][3]) + " of " + m[0])

Santa has to pack:
87 of Teddy Bear
7 of LEGO Saturn V Rocket
6 of Mountain Bike
20 of LEGO Mindstorms RobotKit
10 of Barbie Malibu House
0 of Playmobil RC Freight Train


## Task 2. ILP Solution

<b>Your task:</b> Solve the problem with `pulp` by creating and solving an integer linear program.

In [2]:
import pulp

integerProgram = pulp.LpProblem("Knapsack_Integer_Problem", pulp.LpMaximize)

#Define the variable
Vars = pulp.LpVariable.dicts("Number_of",(k[0] for k in items.items()), lowBound=0, upBound=100, cat=pulp.LpInteger)

#Change the upper bound to the correct value
for k in items.items():
    Vars[k[0]].upBound = k[1][2]
    
# Objective Function
integerProgram += pulp.lpSum([m[1][1] * Vars[m[0]] for m in items.items()])

# Constraint
integerProgram += pulp.lpSum([m[1][0] * Vars[m[0]] for m in items.items()]) <=250

integerProgram.solve()

print("Santa has to pack:")
for k in items.items():
    print(str(Vars[k[0]].value()) + " of " + k[0])

Santa has to pack:
87.0 of Teddy Bear
7.0 of LEGO Saturn V Rocket
6.0 of Mountain Bike
20.0 of LEGO Mindstorms RobotKit
10.0 of Barbie Malibu House
0.0 of Playmobil RC Freight Train


## Task 3. Dropping Integrality

<b>Your task:</b> Remove the integrality constraints from the IP you created in Task 2 and solve the resulting LP.

How can you obtain the optimal solution to the LP via a manual calculation, without using `pulp`?

In [3]:
linearProgram = pulp.LpProblem("Knapsack_Linear_Problem", pulp.LpMaximize)

#Define the variables
Vars = pulp.LpVariable.dicts("Number_of",(k[0] for k in items.items()), lowBound=0, upBound=100, cat=pulp.LpContinuous)

#Change the upper bound to the correct value
for k in items.items():
    Vars[k[0]].upBound = k[1][2]
    
# Objective Function
linearProgram += pulp.lpSum([m[1][1] * Vars[m[0]] for m in items.items()])

# Constraint
linearProgram += pulp.lpSum([m[1][0] * Vars[m[0]] for m in items.items()]) <=250

linearProgram.solve()

print("Santa has to pack:")
for k in items.items():
    print(str(Vars[k[0]].value()) + " of " + k[0])

Santa has to pack:
90.0 of Teddy Bear
6.4 of LEGO Saturn V Rocket
6.0 of Mountain Bike
20.0 of LEGO Mindstorms RobotKit
10.0 of Barbie Malibu House
0.0 of Playmobil RC Freight Train


Since the gifts can now be packed fractionally, it's always worth it to take the gift with a higher ratio of happiness to weight over the one with a lower ratio. Therefore, we can obtain the optimal solution to the LP as follows:

1. Compute the happiness to weight ratio for every gift;
2. Pack whole gifts in the order of decreasing happiness per kilogram as long as we don't exceed the weight limit;
3. If taking the last gift would exceed the weight limit, take a fraction of it that would get us exactly to the weight limit.

This solution is optimal, since we can replace a fraction of a taken gift with lower happiness per kilogram with the same fraction of a gift not taken with higher happiness to weight ratio.